In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import numpy as np, healpy as hp, scipy.stats
import h5py, tqdm
import scipy

In [3]:
sys.path.append('/home/andy/Documents/Projects/EDR3_selection/Gaia-EDR3-selection-function/Python/')
from GenerateSphHarm_decomposed import sphharm_sum_quick, sphharm_sum_truth

# Runtime tests

In [4]:
nside=128; lmax=85
data = {}
with h5py.File('/data/asfe2/Projects/gaia_edr3/sphericalharmonics_decomposed_nside{0}_lmax{1}.h5'.format(nside,lmax), 'r') as hf:
    for key in hf.keys():
        data[key] = hf[key][...]

In [5]:
alm = np.random.rand(len(data['m']))

In [6]:
Nring = data['lambda'].shape[1]
F = np.zeros((2*lmax+1,Nring))

In [7]:
%time sphharm_sum_quick(alm, data['lambda'], data['azimuth'], data['m'],\
                        lmax, data['jpix'], Nring, F)

CPU times: user 944 ms, sys: 0 ns, total: 944 ms
Wall time: 946 ms


10863.915126475224

In [8]:
%timeit sphharm_sum_quick(alm, data['lambda'], data['azimuth'], data['m'],\
                        lmax, data['jpix'], Nring, F)

459 ms ± 12.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%time sphharm_sum_truth(alm, nside, lmax)

100%|██████████| 961/961 [01:37<00:00,  9.89it/s]

CPU times: user 1min 36s, sys: 208 ms, total: 1min 37s
Wall time: 1min 37s


26341.65389367123

# Spherical harmonic recurrence

In [95]:
nside = 1
lmax = 5
Npix = hp.nside2npix(nside)

# Form the l's and m's
l = np.concatenate([np.repeat(_l,2*_l+1) for _l in range(lmax+1)]).astype(np.int)
m = np.concatenate([np.arange(-_l,_l+1e-9) for _l in range(lmax+1)]).astype(np.int)
l_hp,m_hp = hp.sphtfunc.Alm.getlm(lmax=lmax)
Nmodes = int((lmax+1)**2)
Nmodes_healpy = int((lmax+1)*(lmax+2)/2)

# Ring idxs of pixels with phi=0
theta, phi = hp.pix2ang(nside, np.arange(hp.nside2npix(nside)))
theta_ring, jpix = np.unique(theta, return_inverse=True)

# Generate Lambda
alm_hp = np.zeros(Nmodes_healpy)
_lambda_true = np.zeros((Nmodes, 4*nside-1))
for i,(_l,_m) in enumerate(zip(tqdm.tqdm(l),m)):
    _lambda_true[i] = (-1)**np.abs(_m) * np.real( scipy.special.sph_harm(np.abs(_m), _l, theta_ring*0., theta_ring) )

100%|██████████| 36/36 [00:00<00:00, 19668.48it/s]


In [96]:
_lambda_true

array([[ 2.82094792e-01,  2.82094792e-01,  2.82094792e-01],
       [ 2.57516135e-01,  3.45494149e-01,  2.57516135e-01],
       [ 3.25735008e-01, -0.00000000e+00, -3.25735008e-01],
       [ 2.57516135e-01,  3.45494149e-01,  2.57516135e-01],
       [ 2.14596779e-01,  3.86274202e-01,  2.14596779e-01],
       [ 3.83882388e-01, -0.00000000e+00, -3.83882388e-01],
       [ 1.05130522e-01, -3.15391565e-01,  1.05130522e-01],
       [ 3.83882388e-01, -0.00000000e+00, -3.83882388e-01],
       [ 2.14596779e-01,  3.86274202e-01,  2.14596779e-01],
       [ 1.72766821e-01,  4.17223824e-01,  1.72766821e-01],
       [ 3.78513139e-01, -0.00000000e+00, -3.78513139e-01],
       [ 2.94414128e-01, -3.23180184e-01,  2.94414128e-01],
       [-1.93498839e-01, -3.80841001e-17,  1.93498839e-01],
       [ 2.94414128e-01, -3.23180184e-01,  2.94414128e-01],
       [ 3.78513139e-01, -0.00000000e+00, -3.78513139e-01],
       [ 1.72766821e-01,  4.17223824e-01,  1.72766821e-01],
       [ 1.36584164e-01,  4.42532692e-01

In [59]:
Nring = theta_ring.shape[0]

In [132]:
# Generate Lambda
_lambda = np.zeros((len(l), 4*nside-1))
lval = -1
x = np.cos(theta_ring)
for i,(_l,_m) in enumerate(zip(l,m)):
    if _l>lval:
        lval = _l
        if _l>1: _lambda_prev2 = _lambda_prev.copy()
        if _l>0: 
            _lambda_prev = _lambda_now.copy()
        _lambda_now = np.zeros((2*lval+1, Nring))

    if _l<3:
        _lambda_now[_m+_l] = (-1)**np.abs(_m) * np.real( scipy.special.sph_harm(np.abs(_m), _l, theta_ring*0., theta_ring) )
    else:
        if _m==-_l:
            _lambda_now[_m+_l] = -_lambda_prev[(_m+1)+(_l-1)] * np.sqrt((2*_l+1)/(2*_l))/(2*_l-1) * np.sqrt(1-x**2)/(2*_l)
        elif _m==-_l+1:
            _lambda_now[_m+_l] = x*_lambda_prev[_m+(_l-1)]
        if np.abs(_m)<_l-1:
            _lambda_now[_m+_l] = 1/(_l-1) * ( np.sqrt((2*_l+1)/(2*_l-1) * (_l-np.abs(_m))/(_l+np.abs(_m))) \
                                                                     * _lambda_prev[_m+_l-1] \
                                            + np.sqrt((2*_l+1)/(2*_l-3) * (_l-np.abs(_m))/(_l+np.abs(_m)) 
                                                                     * (_l-np.abs(_m)-1)/(_l+np.abs(_m)-1)) 
                                                                     * _lambda_prev2[_m+_l-2] )
        elif _m==_l-1:
            _lambda_now[_m+_l] = _lambda_prev[_m+(_l-1)] * np.sqrt(2*_l+1)*x
        elif _m==_l:
            _lambda_now[_m+_l] = _lambda_prev[_m+_l-2] * np.sqrt((2*_l+1)/(2*_l))*np.sqrt(1-x**2)

            
    _lambda[i] = _lambda_now[_m+_l]

In [133]:
list(zip(np.abs(_lambda-_lambda_true)[:,0], l, m))

[(0.0, 0, 0),
 (0.0, 1, -1),
 (0.0, 1, 0),
 (0.0, 1, 1),
 (0.0, 2, -2),
 (0.0, 2, -1),
 (0.0, 2, 0),
 (0.0, 2, 1),
 (0.0, 2, 2),
 (0.17852571466502434, 3, -3),
 (0.23544862015188614, 3, -2),
 (0.05352984153038051, 3, -1),
 (0.5044791162792299, 3, 0),
 (0.05352984153038051, 3, 1),
 (5.551115123125783e-17, 3, 2),
 (5.551115123125783e-17, 3, 3),
 (0.1365028642370528, 4, -4),
 (0.3493729039681121, 4, -3),
 (0.33634336970069867, 4, -2),
 (0.13843540707421811, 4, -1),
 (0.5263157457633001, 4, 0),
 (0.13843540707421811, 4, 1),
 (0.2849642685521785, 4, 2),
 (1.6653345369377348e-16, 4, 3),
 (2.7755575615628914e-17, 4, 4),
 (0.10677345899209897, 5, -5),
 (0.3019447502034608, 5, -4),
 (0.4306236713451623, 5, -3),
 (0.18215266333390268, 5, -2),
 (0.34128668361799597, 5, -1),
 (0.4288178489958862, 5, 0),
 (0.34128668361799597, 5, 1),
 (0.1449671999791066, 5, 2),
 (0.37176962209768916, 5, 3),
 (5.551115123125783e-17, 5, 4),
 (2.7755575615628914e-17, 5, 5)]

In [102]:
l, m

(array([0, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]),
 array([ 0, -1,  0,  1, -2, -1,  0,  1,  2, -3, -2, -1,  0,  1,  2,  3, -4,
        -3, -2, -1,  0,  1,  2,  3,  4, -5, -4, -3, -2, -1,  0,  1,  2,  3,
         4,  5]))

In [22]:
_lambda_now.shape

(1, 31)

In [21]:
_lambda.shape

(121, 31)

In [18]:
theta_ring.shape

(31,)